## Basic Model


After running `00_Preprocess.ipynb` you can run this notebook to train a single model. This is mostly used for quick experiments (comparing models, normalizing with different image stats etc.) and not for full training.

For full training code see `src/trainAll.py`

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import shutil
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from fastai.vision import Path, ImageList, imagenet_stats, cnn_learner, get_transforms, DatasetType, models, load_learner, fbeta
import sklearn.metrics
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
np.random.seed(42)
torch.manual_seed(0)

In [16]:
NFOLDS = 5
script_name = os.path.basename('01_BasicModel').split('.')[0]
MODEL_NAME = "{0}__folds{1}".format(script_name, NFOLDS)
print("Model: {}".format(MODEL_NAME))

# Make required folders if they're not already present
directories = ['kfolds', 'model_predictions', 'model_source']
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

Model: 01_BasicModel__folds5


In [17]:
def calculate_overall_lwlrap_sklearn(scores, truth):
    # Calculate the overall lwlrap using sklearn.metrics.lrap.
    # sklearn doesn't correctly apply weighting to samples with no labels, so just skip them.
    overall_lwlrap = sklearn.metrics.label_ranking_average_precision_score(truth > 0, scores)
    
    return torch.Tensor([overall_lwlrap])

In [18]:
DATA = Path('data')
WORK = Path('work')

CSV_TRN_MERGED = DATA/'train_merged.csv'
CSV_SUBMISSION = DATA/'sample_submission.csv'

TRN_CURATED = DATA/'train_curated2'
TRN_NOISY = DATA/'train_noisy2'

IMG_TRN_CURATED = WORK/'image/trn_curated2'
IMG_TRN_NOISY = WORK/'image/trn_noisy2'
IMG_TEST = WORK/'image/test'

TEST = DATA/'test'

train = pd.read_csv(DATA/'train_curated.csv')
test = pd.read_csv(DATA/'sample_submission.csv')
train_noisy = pd.read_csv(DATA/'train_noisy.csv')
train_merged = pd.read_csv(DATA/'train_merged.csv')

In [19]:
X = train['fname']
y = train['labels'].apply(lambda f: f.split(','))
y_noisy = train_noisy['labels'].apply(lambda f: f.split(','))
transformed_y = MultiLabelBinarizer().fit_transform(y)
transformed_y_noisy = MultiLabelBinarizer().fit_transform(y_noisy)
filenames = train['fname'].values
filenames = filenames.reshape(-1, 1)

oof_preds = np.zeros((len(train), 80))
test_preds = np.zeros((len(test), 80))

tfms = get_transforms(do_flip=True, max_rotate=0, max_lighting=0.1, max_zoom=0, max_warp=0.)

mskf = MultilabelStratifiedKFold(n_splits=5, random_state=4, shuffle=True)
_, val_index = next(mskf.split(X, transformed_y))

In [20]:
#Our clasifier stuff    
src = (ImageList.from_csv(WORK/'image', Path('../../')/CSV_TRN_MERGED, folder='trn_merged2', suffix='.jpg')
    .split_by_idx(val_index)
    .label_from_df(cols=list(train_merged.columns[1:]))
      )

data = (src.transform(tfms, size=128).databunch(bs=64).normalize())

In [21]:
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(data, models.vgg16_bn, pretrained=False, metrics=[f_score]).mixup(stack_y=False)
learn.fit_one_cycle(100, 1e-2)

epoch,train_loss,valid_loss,fbeta,time
0,0.131549,0.085804,0.000000,01:07
1,0.108097,0.075875,0.000000,01:09
2,0.106192,0.099027,0.000555,01:05
3,0.105002,0.069065,0.015152,01:08
4,0.103803,0.070100,0.022576,01:08
5,0.102333,0.071095,0.025433,01:09
6,0.100219,0.060648,0.096153,01:09
7,0.099529,0.060368,0.104957,01:09
8,0.099372,0.056387,0.146524,01:09
9,0.098162,0.056575,0.174415,01:09
